In [49]:
import pandas as pd
import os
import tarfile
from PIL import Image
import torchvision 
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset

from datetime import datetime
#Extracting solar image data.
file = tarfile.open('solar_Flare.tar.gz') 
  
file.extractall('c:\\Users\\Shiyan Liu\\Github\\Solar_Flare_CNN') 
  
file.close() 

#Reading flare dataset.
flareDataset=pd.read_csv('flareDataset.csv')
num_Rows, num_Columns=flareDataset.shape
solar_Images=[f for f in os.listdir("img") if f.endswith(".jpg")]




In [42]:
#Loading images
transformed_Images=[]
for imageFilePath in solar_Images:
    image=Image.open("img\\"+imageFilePath)
    tensor_Transform=torchvision.transforms.ToTensor()
    transformed_Image=tensor_Transform(image)
    transformed_Images.append(transformed_Image)
    
images_Dataset=TensorDataset(transformed_Image)

In [50]:
#Setting everything to a universal time scale, with the 0 of time being the first image. 
initial_Time_Split=solar_Images[0].split("_")
date=initial_Time_Split[0]
time=initial_Time_Split[1]
s=""
s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
s+=" "
s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
t0=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

deltat_Images=[]

for imageName in solar_Images:
    s_split=imageName.split("_")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
    s+=" "
    s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    deltat_Images.append((t-t0).days)
    
deltat_Flares=[]
for i in range(num_Rows):
    s_split=flareDataset.loc[i, "T_REC"].split(" ")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=date
    s+=" "
    s+=time
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    flareDataset.loc[i, "T_REC"]=(t-t0).days
    
for i in range(num_Rows):
    if int(flareDataset.loc[i,"T_REC"])<0 or int(flareDataset.loc[i, "FlareNumber"]) !=1:
        flareDataset.drop(i, inplace=True)

print(flareDataset)

      FlareNumber T_REC  NOAA_AR  QUALITY  Longitude  Latitude   TOTUSJH  \
8494            1    65    11093        0         45        12   956.465   
8495            1   135    11112     1024         15        19   837.486   
8496            1   255    11158        0         11        19   488.687   
8497            1   256    11158        0          3        20  2624.992   
8498            1   256    11158        0         14        21  2887.519   
...           ...   ...      ...      ...        ...       ...       ...   
8869            1  2654    12673        0         58         9  5769.701   
8870            1  2654    12673        0         58         9  5522.014   
8871            1  2655    12673        0         70         9  5562.046   
8872            1  2655    12673        0         70         9  5072.454   
8873            1  2655    12673        0         70         9  4000.937   

            TOTBSQ        TOTPOT       TOTUSJZ  ...  MEANGBT  MEANGBZ  \
8494  2.291500

In [51]:
#Residual neural network for regression.

weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
resnet=torchvision.models.resnet18(weights)

#Replacing classification layer with regression layer.
resnet.fc=nn.Linear(512, 1, bias=False)




      FlareNumber T_REC  NOAA_AR  QUALITY  Longitude  Latitude   TOTUSJH  \
8494            1    65    11093        0         45        12   956.465   
8495            1   135    11112     1024         15        19   837.486   
8496            1   255    11158        0         11        19   488.687   
8497            1   256    11158        0          3        20  2624.992   
8498            1   256    11158        0         14        21  2887.519   
...           ...   ...      ...      ...        ...       ...       ...   
8869            1  2654    12673        0         58         9  5769.701   
8870            1  2654    12673        0         58         9  5522.014   
8871            1  2655    12673        0         70         9  5562.046   
8872            1  2655    12673        0         70         9  5072.454   
8873            1  2655    12673        0         70         9  4000.937   

            TOTBSQ        TOTPOT       TOTUSJZ  ...  MEANGBT  MEANGBZ  \
8494  2.291500

C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
